<a href="https://colab.research.google.com/github/CrisEsp/Heur-stico/blob/main/Heuristico_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import time
import threading
import sys

# Definición de capacidades de tolvas y niveles iniciales
tolvas_MC1 = {
    "Clinker": {"capacidad": 500, "max_metros": 14},
    "Puzolana": {"capacidad": 300, "max_metros": 12},
    "Yeso": {"capacidad": 300, "max_metros": 10}
}
tolvas_MC2 = {
    "Clinker": {"capacidad": 300, "max_metros": 9},
    "Puzolana_Humeda": {"capacidad": 500, "max_metros": 15, "tolva": "426HO04"},
    "Puzolana_Seca": {"capacidad": 100, "max_metros": 12, "tolva": "426HO02"},
    "Yeso": {"capacidad": 120, "max_metros": 9}
}
tolvas_MC3 = {
    "Clinker": {"capacidad": 60, "max_porcentaje": 100},
    "Clinker_Silo_Blanco": {"capacidad": 500, "max_metros": 10.5},
    "Puzolana": {"capacidad": 35, "max_porcentaje": 100},
    "Yeso": {"capacidad": 30, "max_porcentaje": 100}
}

niveles_MC1 = {"Clinker": 0, "Puzolana": 0, "Yeso": 0}
niveles_MC2 = {"Clinker": 0, "Puzolana_Humeda": 0, "Puzolana_Seca": 0, "Yeso": 0}
niveles_MC3 = {"Clinker": 0, "Clinker_Silo_Blanco": 0, "Puzolana": 0, "Yeso": 0}

# Variables globales para rastrear alimentaciones y temporizadores
alimentaciones_actuales = set()
alimentaciones_en_progreso = {}
temporizadores_molinos = {}
lock = threading.Lock()
lock_print = threading.Lock()

def resetear_niveles_molino(molino):
    global niveles_MC1, niveles_MC2, niveles_MC3
    if molino == "MC1":
        niveles_MC1 = {"Clinker": 0, "Puzolana": 0, "Yeso": 0}
    elif molino == "MC2":
        niveles_MC2 = {"Clinker": 0, "Puzolana_Humeda": 0, "Puzolana_Seca": 0, "Yeso": 0}
    elif molino == "MC3":
        niveles_MC3 = {"Clinker": 0, "Clinker_Silo_Blanco": 0, "Puzolana": 0, "Yeso": 0}

def ajustar_velocidad(material, min_vel, max_vel):
    while True:
        try:
            velocidad = float(input(f" Ingrese la velocidad del transportador para {material} ({min_vel}-{max_vel} RPM): "))
            if min_vel <= velocidad <= max_vel:
                print(f" Ajustando velocidad del transportador a {velocidad:.2f} RPM para {material}")
                return velocidad
            else:
                print(f"La velocidad debe estar entre {min_vel} y {max_vel} RPM.")
        except ValueError:
            print("Por favor, ingrese un número válido.")

def verificar_nivel_yeso(molino, nivel_actual):
    if molino == "MC1" and nivel_actual > 2:
        print(f"Advertencia: El nivel de yeso en {molino} ({nivel_actual:.2f}m) excede el límite de 2m.")
        while True:
            try:
                nuevo_nivel = float(input("Ingrese el nuevo nivel de yeso (debe ser 2m o menos): "))
                if nuevo_nivel <= 2:
                    return nuevo_nivel
                else:
                    print("El nivel debe ser 2m o menos.")
            except ValueError:
                print("Por favor, ingrese un número válido.")
    return nivel_actual

def validar_restricciones(molino, material):
    global alimentaciones_actuales, alimentaciones_en_progreso

    with lock:
        # Verificar si ya se está alimentando el mismo material al mismo molino
        if molino in alimentaciones_en_progreso and material in alimentaciones_en_progreso[molino]:
            return False, f"Ya se está alimentando {material} a {molino}."

        # Restricciones para Clinker
        if material == "Clinker":
            if any(alim[1] == "Clinker" for alim in alimentaciones_actuales):
                return False, "No se puede alimentar Clinker a múltiples molinos simultáneamente"

        # Restricciones para MC2
        if molino == "MC2":
            materiales_actuales = [alim[1] for alim in alimentaciones_actuales if alim[0] == "MC2"]
            if material in ["Puzolana_Humeda", "Puzolana_Seca"] and any(m in materiales_actuales for m in ["Puzolana_Humeda", "Puzolana_Seca"]):
                return False, "No se puede alimentar dos tipos de Puzolana simultáneamente a MC2"
            if material == "Yeso" and "Yeso" in materiales_actuales:
                return False, "Ya se está alimentando Yeso a MC2"

        # Restricciones para MC3
        if molino == "MC3":
            materiales_actuales = [alim[1] for alim in alimentaciones_actuales if alim[0] == "MC3"]
            if len(materiales_actuales) >= 2 and material not in materiales_actuales:
                return False, "No se pueden alimentar más de dos materiales a MC3 simultáneamente"
            if "Clinker" in materiales_actuales and material in ["Puzolana", "Yeso"]:
                return False, "No se puede alimentar Clinker y Puzolana/Yeso a MC3 simultáneamente"

        # Restricciones para MC1
        if molino == "MC1":
            if material in ["Puzolana", "Yeso"] and "Puzolana" in alimentaciones_en_progreso.get(molino, []) and "Yeso" in alimentaciones_en_progreso.get(molino, []):
                return False, "No se puede alimentar Puzolana y Yeso simultáneamente a MC1"

        # Restricción general para Yeso en MC1
        if molino == "MC1" and material == "Yeso":
            if niveles_MC1["Yeso"] + 2 > tolvas_MC1["Yeso"]["max_metros"]:
                return False, "No se puede alimentar más Yeso a MC1, se excedería el límite de 2m"

    return True, "Validación exitosa"

def imprimir_progreso(molino, material, porcentaje):
    barra = f"[{'#' * int(porcentaje / 2)}{' ' * (50 - int(porcentaje / 2))}]"
    with lock_print:
        sys.stdout.write(f"\r{molino} - Alimentando {material}: {barra} {porcentaje:.1f}%")
        sys.stdout.flush()

def simular_alimentacion(molino, material, cantidad, duracion_real):
    tiempo_inicio = time.time()
    tiempo_simulado = 0
    while tiempo_simulado < 60:  # 1 hora simulada
        tiempo_actual = time.time() - tiempo_inicio
        tiempo_simulado = (tiempo_actual / duracion_real) * 60
        porcentaje = min(tiempo_simulado / 60 * 100, 100)
        imprimir_progreso(molino, material, porcentaje)
        time.sleep(0.1)

    with lock_print:
        print(f"\n{molino} - Alimentación de {material} completada.")

    # Actualizar niveles después de la alimentación
    with lock:
        if molino == "MC1":
            niveles_MC1[material] += cantidad
        elif molino == "MC2":
            niveles_MC2[material] += cantidad
        elif molino == "MC3":
            niveles_MC3[material] += cantidad

        if molino in alimentaciones_en_progreso:
            alimentaciones_en_progreso[molino].remove(material)
        if (molino, material) in alimentaciones_actuales:
            alimentaciones_actuales.remove((molino, material))

        print(f"\n{molino} - Alimentación de {material} completada. Ruta habilitada para nueva alimentación.")

    if not alimentaciones_en_progreso.get(molino):
        temporizadores_molinos[molino] = False

def iniciar_alimentacion(molino, material, cantidad):
    if molino not in alimentaciones_en_progreso:
        alimentaciones_en_progreso[molino] = set()
    alimentaciones_en_progreso[molino].add(material)
    temporizadores_molinos[molino] = True
    thread = threading.Thread(target=simular_alimentacion, args=(molino, material, cantidad, 30))
    thread.start()
    print(f"\nIniciando alimentación de {material} a {molino}")

def intentar_alimentar(molino, material, cantidad):
    valido, mensaje = validar_restricciones(molino, material)
    if not valido:
        print(f"Error: No se puede alimentar {material} a {molino}: {mensaje}")
        return 0

    with lock:
        alimentaciones_actuales.add((molino, material))
        if molino not in alimentaciones_en_progreso:
            alimentaciones_en_progreso[molino] = set()
        alimentaciones_en_progreso[molino].add(material)

    if molino == "MC1":
        niveles = niveles_MC1
        tolvas = tolvas_MC1
    elif molino == "MC2":
        niveles = niveles_MC2
        tolvas = tolvas_MC2
    elif molino == "MC3":
        niveles = niveles_MC3
        tolvas = tolvas_MC3

    if material in ["Puzolana", "Puzolana_Humeda", "Puzolana_Seca"]:
        ajustar_velocidad("puzolana", 1100, 1150)
    elif material == "Yeso":
        ajustar_velocidad("yeso", 850, 900)

    capacidad = tolvas[material]['capacidad']
    if 'max_metros' in tolvas[material]:
        max_nivel = tolvas[material]['max_metros']
        nivel_actual = niveles[material] / capacidad * max_nivel
        if molino == "MC1" and material == "Yeso":
            max_nivel = min(max_nivel, 2)  # Limitar a 2m para yeso en MC1
        espacio_disponible = max_nivel - nivel_actual
        cantidad_metros = min(cantidad / capacidad * max_nivel, espacio_disponible)
        cantidad_alimentada = cantidad_metros / max_nivel * capacidad
    else:  # Para MC3 que usa porcentajes
        max_nivel = tolvas[material]['max_porcentaje']
        nivel_actual = niveles[material] / capacidad * 100
        espacio_disponible = max_nivel - nivel_actual
        cantidad_porcentaje = min(cantidad / capacidad * 100, espacio_disponible)
        cantidad_alimentada = cantidad_porcentaje / 100 * capacidad

    if cantidad_alimentada > 0:
        print(f"Iniciando alimentación de {cantidad_alimentada:.2f}t de {material} a {molino}")
        iniciar_alimentacion(molino, material, cantidad_alimentada)
    else:
        print(f"No se pudo alimentar {material} a {molino}. Tolva llena o se alcanzó el límite máximo.")
        with lock:
            alimentaciones_actuales.discard((molino, material))
            if molino in alimentaciones_en_progreso:
                alimentaciones_en_progreso[molino].discard(material)

    return cantidad_alimentada

def todas_tolvas_llenas():
    rendimiento_MC1_MC2 = 0.8
    rendimiento_MC3 = 0.5

    if any(nivel < tolvas_MC1[material]['capacidad'] * rendimiento_MC1_MC2 for material, nivel in niveles_MC1.items()):
        return False
    if any(nivel < tolvas_MC2[material]['capacidad'] * rendimiento_MC1_MC2 for material, nivel in niveles_MC2.items()):
        return False
    if any(nivel < tolvas_MC3[material]['capacidad'] * rendimiento_MC3 for material, nivel in niveles_MC3.items()):
        return False
    return True

def calcular_necesidades(molino, tipo_produccion):
    produccion_MC1 = {
        "P30": (0.30, 0.015, 72),
        "P40": (0.40, 0.015, 64)
    }
    produccion_MC2 = {
        "P10": (0.10, 0.03, 70),
        "P16": (0.16, 0.025, 80),
        "P20": (0.12, 0.025, 87),
        "P30": (0.30, 0.02, 110)
    }
    produccion_MC3 = {
        "P30": (0.30, 0.025, 37)
    }

    if molino == "MC1" and tipo_produccion in produccion_MC1:
        puzolana, yeso, produccion = produccion_MC1[tipo_produccion]
        clinker = 1 - puzolana - yeso
        return {
            "Clinker": clinker * produccion,
            "Puzolana": puzolana * produccion,
            "Yeso": yeso * produccion
        }
    elif molino == "MC2" and tipo_produccion in produccion_MC2:
        puzolana, yeso, produccion = produccion_MC2[tipo_produccion]
        clinker = 1 - puzolana - yeso
        return {
            "Clinker": clinker * produccion,
            "Puzolana_Humeda": puzolana * produccion * 0.7,
            "Puzolana_Seca": puzolana * produccion * 0.3,
            "Yeso": yeso * produccion
        }
    elif molino == "MC3" and tipo_produccion in produccion_MC3:
        puzolana, yeso, produccion = produccion_MC3[tipo_produccion]
        clinker = 1 - puzolana - yeso
        return {
            "Clinker": clinker * produccion,
            "Puzolana": puzolana * produccion,
            "Yeso": yeso * produccion
        }

    print(f"Advertencia: No se encontró configuración para {molino} con producto {tipo_produccion}")
    return {"Clinker": 60, "Puzolana": 30, "Yeso": 10}

def redondear_diccionario(d, decimales=2):
    return {k: round(v, decimales) if isinstance(v, (int, float)) else v for k, v in d.items()}

def calcular_tiempos_llenado(molino, necesidades):
    tiempos_llenado = {}
    if molino == "MC1":
        rendimiento = 0.8
        for mat, cant in necesidades.items():
            if mat == "Yeso":
                capacidad_objetivo = min(tolvas_MC1[mat]["capacidad"] * rendimiento, 2 / tolvas_MC1[mat]["max_metros"] * tolvas_MC1[mat]["capacidad"])
            else:
                capacidad_objetivo = tolvas_MC1[mat]["capacidad"] * rendimiento
            nivel_actual = niveles_MC1[mat]
            if cant > 0:
                tiempos_llenado[mat] = (capacidad_objetivo - nivel_actual) / cant
            else:
                tiempos_llenado[mat] = float('inf')

    elif molino == "MC2":
        rendimiento = 0.8
        for mat, cant in necesidades.items():
            capacidad_objetivo = tolvas_MC2[mat]["capacidad"] * rendimiento
            nivel_actual = niveles_MC2[mat]
            if cant > 0:
                tiempos_llenado[mat] = (capacidad_objetivo - nivel_actual) / cant
            else:
                tiempos_llenado[mat] = float('inf')
    else:  # MC3
        rendimiento = 0.5
        for mat, cant in necesidades.items():
            capacidad_objetivo = tolvas_MC3[mat]["capacidad"] * rendimiento
            nivel_actual = niveles_MC3[mat]
            if cant > 0:
                tiempos_llenado[mat] = (capacidad_objetivo - nivel_actual) / cant
            else:
                tiempos_llenado[mat] = float('inf')

    return tiempos_llenado

def calcular_tiempo_vaciado(molino, material):
    if molino == "MC1":
        niveles = niveles_MC1
        tolvas = tolvas_MC1
    elif molino == "MC2":
        niveles = niveles_MC2
        tolvas = tolvas_MC2
    else:  # MC3
        niveles = niveles_MC3
        tolvas = tolvas_MC3

    # Manejar el caso especial de Puzolana para MC1 y MC3
    if material == "Puzolana" and molino in ["MC1", "MC3"]:
        capacidad = tolvas["Puzolana"]['capacidad']
        nivel_actual = niveles["Puzolana"]
    elif material == "Puzolana_Humeda" and molino == "MC2":
        capacidad = tolvas["Puzolana_Humeda"]['capacidad']
        nivel_actual = niveles["Puzolana_Humeda"]
    elif material == "Puzolana_Seca" and molino == "MC2":
        capacidad = tolvas["Puzolana_Seca"]['capacidad']
        nivel_actual = niveles["Puzolana_Seca"]
    else:
        capacidad = tolvas[material]['capacidad']
        nivel_actual = niveles[material]

    consumo = calcular_consumo(molino, material)

    if consumo > 0:
        return (nivel_actual - nivel_ineficiente(molino, material)) / consumo
    else:
        return float('inf')

def nivel_ineficiente(molino, material):
    if molino in ["MC1", "MC2"]:
        if molino == "MC1":
            return 0.2 * tolvas_MC1[material]['capacidad']  # 20% de la capacidad
        else:  # MC2
            if material in ["Puzolana_Humeda", "Puzolana_Seca"]:
                return 0.2 * tolvas_MC2[material]['capacidad']  # 20% de la capacidad
            else:
                return 0.2 * tolvas_MC2[material]['capacidad']  # 20% de la capacidad
    else:  # MC3
        return 0.5 * tolvas_MC3[material]['capacidad']  # 50% de la capacidad

def calcular_consumo(molino, material):
    tipo_produccion = obtener_tipo_produccion_actual(molino)
    necesidades = calcular_necesidades(molino, tipo_produccion)
    if molino == "MC2" and material in ["Puzolana_Humeda", "Puzolana_Seca"]:
        return necesidades.get("Puzolana_Humeda", 0) + necesidades.get("Puzolana_Seca", 0)
    return necesidades.get(material, 0)

def obtener_tipo_produccion_actual(molino):
    # Esta función debería obtener el tipo de producción actual del molino
    # Por ahora, usaremos un valor predeterminado
    if molino == "MC1":
        return "P30"
    elif molino == "MC2":
        return "P16"
    else:  # MC3
        return "P30"

def seleccionar_ruta_alimentacion(molino, material):
    if molino == "MC1":
        if material == "Clinker":
            return "Hacia MC1 desde Pretrit"
        elif material == "Puzolana":
            tiempo_vaciado_MC1 = calcular_tiempo_vaciado("MC1", "Puzolana")
            tiempo_vaciado_MC2 = calcular_tiempo_vaciado("MC2", "Puzolana_Humeda")
            return "Húmeda a MC1 por MC1" if tiempo_vaciado_MC1 < tiempo_vaciado_MC2 else "Húmeda a MC1 por MC2"
        elif material == "Yeso":
            tiempo_vaciado_MC1 = calcular_tiempo_vaciado("MC1", "Yeso")
            tiempo_vaciado_MC2 = calcular_tiempo_vaciado("MC2", "Yeso")
            return "Hacia MC1 por MC1" if tiempo_vaciado_MC1 < tiempo_vaciado_MC2 else "Hacia MC1 por MC2"
    elif molino == "MC2":
        if material == "Clinker":
            return "Hacia MC2 desde Pretrit"
        elif material == "Puzolana_Humeda":
            return "P.H. a 426HO04 por MC2"
        elif material == "Puzolana_Seca":
            return "P.S a 426HO02 por 426HO04"
        elif material == "Yeso":
            return "Hacia MC2 por MC2"
    elif molino == "MC3":
        if material == "Clinker":
            if niveles_MC3["Clinker_Silo_Blanco"] / tolvas_MC3["Clinker_Silo_Blanco"]["capacidad"] > 0.3:
                return "Hacia MC3 desde Silo3"
            else:
                return "Hacia Silo 3 desde Pretrit"
        elif material == "Puzolana":
            return "P. seca a MC3 por MC2"
        elif material == "Yeso":
            tiempo_vaciado_MC1 = calcular_tiempo_vaciado("MC1", "Yeso")
            tiempo_vaciado_MC2 = calcular_tiempo_vaciado("MC2", "Yeso")
            tiempo_vaciado_MC3 = calcular_tiempo_vaciado("MC3", "Yeso")
            if tiempo_vaciado_MC1 <= tiempo_vaciado_MC2 and tiempo_vaciado_MC1 <= tiempo_vaciado_MC3:
                return "Hacia MC3 por MC1"
            elif tiempo_vaciado_MC2 <= tiempo_vaciado_MC1 and tiempo_vaciado_MC2 <= tiempo_vaciado_MC3:
                return "Hacia MC3 por MC2"
            else:
                return "Hacia MC3 por MC3"
    return "Ruta no especificada"

def imprimir_tiempos(molino, tipo_produccion):
    necesidades = calcular_necesidades(molino, tipo_produccion)
    tiempos_llenado = calcular_tiempos_llenado(molino, necesidades)

    print(f"\nTiempos de llenado y vaciado para {molino} - {tipo_produccion}:")
    print("-" * 60)
    print(f"{'Material':<15}{'Tiempo Llenado (h)':<20}{'Tiempo Vaciado (h)':<20}")
    print("-" * 60)

    for material, tiempo_llenado in tiempos_llenado.items():
        tiempo_vaciado = calcular_tiempo_vaciado(molino, material)
        tiempo_llenado_str = f"{tiempo_llenado:.2f}" if tiempo_llenado != float('inf') else "∞"
        tiempo_vaciado_str = f"{tiempo_vaciado:.2f}" if tiempo_vaciado != float('inf') else "∞"
        print(f"{material:<15}{tiempo_llenado_str:<20}{tiempo_vaciado_str:<20}")

def generar_recomendaciones(molino_actual):
    recomendaciones = []
    restricciones = []

    def espacio_disponible(nivel_actual, capacidad, rendimiento):
        return capacidad * rendimiento - nivel_actual

    if molino_actual == "MC1":
        rendimiento = 0.8
        esp_clinker = espacio_disponible(niveles_MC1["Clinker"], tolvas_MC1["Clinker"]["capacidad"], rendimiento)
        esp_puzolana = espacio_disponible(niveles_MC1["Puzolana"], tolvas_MC1["Puzolana"]["capacidad"], rendimiento)
        esp_yeso = min(espacio_disponible(niveles_MC1["Yeso"], tolvas_MC1["Yeso"]["capacidad"], rendimiento), 60 - niveles_MC1["Yeso"])

        if esp_clinker > 0:
            recomendaciones.append(f"Recomiendo alimentar Clinker a MC1 desde Pretrit (Espacio disponible: {esp_clinker:.2f}t)")
        if esp_puzolana > 0:
            recomendaciones.append(f"Recomiendo alimentar Puzolana a MC1 (Espacio disponible: {esp_puzolana:.2f}t)")
        if esp_yeso > 0:
            recomendaciones.append(f"Recomiendo alimentar Yeso a MC1 (Espacio disponible: {esp_yeso:.2f}t)")

    elif molino_actual == "MC2":
        rendimiento = 0.8
        esp_clinker = espacio_disponible(niveles_MC2["Clinker"], tolvas_MC2["Clinker"]["capacidad"], rendimiento)
        esp_puzolana_h = espacio_disponible(niveles_MC2["Puzolana_Humeda"], tolvas_MC2["Puzolana_Humeda"]["capacidad"], rendimiento)
        esp_puzolana_s = espacio_disponible(niveles_MC2["Puzolana_Seca"], tolvas_MC2["Puzolana_Seca"]["capacidad"], rendimiento)
        esp_yeso = espacio_disponible(niveles_MC2["Yeso"], tolvas_MC2["Yeso"]["capacidad"], rendimiento)

        if esp_clinker > 0:
            recomendaciones.append(f"Recomiendo alimentar Clinker a MC2 desde Pretrit (Espacio disponible: {esp_clinker:.2f}t)")
        if esp_puzolana_h > 0:
            recomendaciones.append(f"Recomiendo alimentar Puzolana Húmeda a 426HO04 por MC2 (Espacio disponible: {esp_puzolana_h:.2f}t)")
        if esp_puzolana_s > 0:
            recomendaciones.append(f"Recomiendo alimentar Puzolana Seca a 426HO02 por 426HO04 (Espacio disponible: {esp_puzolana_s:.2f}t)")
        if esp_yeso > 0:
            recomendaciones.append(f"Recomiendo alimentar Yeso a MC2 por MC2 (Espacio disponible: {esp_yeso:.2f}t)")

    elif molino_actual == "MC3":
        rendimiento = 0.5
        esp_clinker = espacio_disponible(niveles_MC3["Clinker"], tolvas_MC3["Clinker"]["capacidad"], rendimiento)
        esp_clinker_sb = espacio_disponible(niveles_MC3["Clinker_Silo_Blanco"], tolvas_MC3["Clinker_Silo_Blanco"]["capacidad"], rendimiento)
        esp_puzolana = espacio_disponible(niveles_MC3["Puzolana"], tolvas_MC3["Puzolana"]["capacidad"], rendimiento)
        esp_yeso = espacio_disponible(niveles_MC3["Yeso"], tolvas_MC3["Yeso"]["capacidad"], rendimiento)

        if esp_clinker > 0:
            if niveles_MC3["Clinker_Silo_Blanco"] / tolvas_MC3["Clinker_Silo_Blanco"]["capacidad"] > 0.3:
                recomendaciones.append(f"Recomiendo alimentar Clinker a MC3 desde Silo Blanco (Espacio disponible: {esp_clinker:.2f}t)")
            else:
                recomendaciones.append(f"Recomiendo alimentar Clinker a Silo Blanco desde Pretrit (Espacio disponible: {esp_clinker_sb:.2f}t)")
        if esp_puzolana > 0:
            recomendaciones.append(f"Recomiendo alimentar Puzolana Seca a MC3 por MC2 (Espacio disponible: {esp_puzolana:.2f}t)")
        if esp_yeso > 0:
            recomendaciones.append(f"Recomiendo alimentar Yeso a MC3 (Espacio disponible: {esp_yeso:.2f}t)")

    if not recomendaciones:
        recomendaciones.append(f"Todas las tolvas de {molino_actual} están llenas o cerca de su capacidad máxima.")

    restricciones.append("Es necesario vaciar el sistema antes de una nueva alimentación para evitar contaminación.")
    restricciones.append("No se puede alimentar simultáneamente Clinker, Puzolana y Yeso a MC3.")
    restricciones.append("No se puede alimentar Clinker a múltiples molinos simultáneamente.")

    return recomendaciones, restricciones

def optimizar_alimentacion(molino, tipo_produccion):
    necesidades = calcular_necesidades(molino, tipo_produccion)
    print(f"\nNecesidades para {molino} - {tipo_produccion}:")
    for material, cantidad in necesidades.items():
        print(f"{material}: {cantidad:.2f} t/h")

    imprimir_tiempos(molino, tipo_produccion)

    recomendaciones, restricciones = generar_recomendaciones(molino)
    print("\nRecomendaciones:")
    for recomendacion in recomendaciones:
        print(recomendacion)

    materiales_alimentados = []
    for material, cantidad_necesaria in necesidades.items():
        if cantidad_necesaria > 0:
            ruta = seleccionar_ruta_alimentacion(molino, material)
            print(f"\nIntentando alimentar {material} a {molino} por la ruta: {ruta}")
            cantidad_alimentada = intentar_alimentar(molino, material, cantidad_necesaria)
            if cantidad_alimentada > 0:
                materiales_alimentados.append(material)

    print("\nRestricciones aplicables:")
    restricciones_aplicables = [r for r in restricciones if any(mat in r for mat in materiales_alimentados)]
    for restriccion in restricciones_aplicables:
        print(restriccion)

    print("\nTiempos actualizados después de la alimentación:")
    imprimir_tiempos(molino, tipo_produccion)

def main():
    global niveles_MC1, niveles_MC2, niveles_MC3
    ciclo = 1

    while True:
        print(f"\n--- Ciclo de optimización {ciclo} ---")

        for molino in ["MC1", "MC2", "MC3"]:
            if molino == "MC1":
                tipos_validos = ["P30", "P40"]
            elif molino == "MC2":
                tipos_validos = ["P10", "P16", "P20", "P30"]
            elif molino == "MC3":
                tipos_validos = ["P30"]

            print(f"\nOptimizando {molino}")
            print(f"Este molino puede producir: {', '.join(tipos_validos)}")
            tipo_produccion = input(f"Ingrese el tipo de producción para {molino} (o presione Enter para omitir): ").upper()

            if tipo_produccion:
                if tipo_produccion not in tipos_validos:
                    print(f"Error, Ingrese un tipo de producción válido para {molino}")
                    continue
                optimizar_alimentacion(molino, tipo_produccion)

        print("\nEstado actual de las alimentaciones:")
        for molino, materiales in alimentaciones_en_progreso.items():
            if materiales:
                print(f"{molino}: {', '.join(materiales)}")
            else:
                print(f"{molino}: Sin alimentaciones en progreso")

        # Esperar a que todos los temporizadores terminen
        while any(temporizadores_molinos.values()):
            time.sleep(1)

        print("\nTodas las alimentaciones han sido completadas.")
        print("\nEstado actual después de la alimentación:")
        print(f"Niveles MC1: {redondear_diccionario(niveles_MC1)}")
        print(f"Niveles MC2: {redondear_diccionario(niveles_MC2)}")
        print(f"Niveles MC3: {redondear_diccionario(niveles_MC3)}")

        if todas_tolvas_llenas():
            print("Todas las tolvas están llenas. Finalizando la simulación.")
            break

        ciclo += 1

        continuar = input("¿Desea continuar con otro ciclo? (s/n): ").lower()
        if continuar != 's':
            break

    print("Simulación completada.")

if __name__ == "__main__":
    main()


--- Ciclo de optimización 1 ---

Optimizando MC1
Este molino puede producir: P30, P40
Ingrese el tipo de producción para MC1 (o presione Enter para omitir): P30

Necesidades para MC1 - P30:
Clinker: 49.32 t/h
Puzolana: 21.60 t/h
Yeso: 1.08 t/h

Tiempos de llenado y vaciado para MC1 - P30:
------------------------------------------------------------
Material       Tiempo Llenado (h)  Tiempo Vaciado (h)  
------------------------------------------------------------
Clinker        8.11                -2.03               
Puzolana       11.11               -2.78               
Yeso           55.56               -55.56              

Recomendaciones:
Recomiendo alimentar Clinker a MC1 desde Pretrit (Espacio disponible: 400.00t)
Recomiendo alimentar Puzolana a MC1 (Espacio disponible: 240.00t)
Recomiendo alimentar Yeso a MC1 (Espacio disponible: 60.00t)

Intentando alimentar Clinker a MC1 por la ruta: Hacia MC1 desde Pretrit
Iniciando alimentación de 49.32t de Clinker a MC1
MC1 - Alimentand

KeyboardInterrupt: Interrupted by user

In [ ]:
# codigo bien ; fecha:11/09/24
import random
import time
import threading
import sys

# Definición de capacidades de tolvas y niveles iniciales
tolvas_MC1 = {
    "Clinker": {"capacidad": 500, "max_metros": 14},
    "Puzolana": {"capacidad": 300, "max_metros": 12},
    "Yeso": {"capacidad": 300, "max_metros": 10}
}
tolvas_MC2 = {
    "Clinker": {"capacidad": 300, "max_metros": 9},
    "Puzolana_Humeda": {"capacidad": 500, "max_metros": 15, "tolva": "426HO04"},
    "Puzolana_Seca": {"capacidad": 100, "max_metros": 12, "tolva": "426HO02"},
    "Yeso": {"capacidad": 120, "max_metros": 9}
}
tolvas_MC3 = {
    "Clinker": {"capacidad": 60, "max_porcentaje": 100},
    "Clinker_Silo_Blanco": {"capacidad": 500, "max_metros": 10.5},
    "Puzolana": {"capacidad": 35, "max_porcentaje": 100},
    "Yeso": {"capacidad": 30, "max_porcentaje": 100}
}

niveles_MC1 = {"Clinker": 0, "Puzolana": 0, "Yeso": 0}
niveles_MC2 = {"Clinker": 0, "Puzolana_Humeda": 0, "Puzolana_Seca": 0, "Yeso": 0}
niveles_MC3 = {"Clinker": 0, "Clinker_Silo_Blanco": 0, "Puzolana": 0, "Yeso": 0}

# Variables globales para rastrear alimentaciones y temporizadores
alimentaciones_actuales = set()
alimentaciones_en_progreso = {}
temporizadores_molinos = {}
lock = threading.Lock()
lock_print = threading.Lock()

def resetear_niveles_molino(molino):
    global niveles_MC1, niveles_MC2, niveles_MC3
    if molino == "MC1":
        niveles_MC1 = {"Clinker": 0, "Puzolana": 0, "Yeso": 0}
    elif molino == "MC2":
        niveles_MC2 = {"Clinker": 0, "Puzolana_Humeda": 0, "Puzolana_Seca": 0, "Yeso": 0}
    elif molino == "MC3":
        niveles_MC3 = {"Clinker": 0, "Clinker_Silo_Blanco": 0, "Puzolana": 0, "Yeso": 0}

def ajustar_velocidad(material, min_vel, max_vel):
    while True:
        try:
            velocidad = float(input(f" Ingrese la velocidad del transportador para {material} ({min_vel}-{max_vel} RPM): "))
            if min_vel <= velocidad <= max_vel:
                print(f" Ajustando velocidad del transportador a {velocidad:.2f} RPM para {material}")
                return velocidad
            else:
                print(f"La velocidad debe estar entre {min_vel} y {max_vel} RPM.")
        except ValueError:
            print("Por favor, ingrese un número válido.")

def verificar_nivel_yeso(molino, nivel_actual):
    if molino == "MC1" and nivel_actual > 2:
        print(f"Advertencia: El nivel de yeso en {molino} ({nivel_actual:.2f}m) excede el límite de 2m.")
        while True:
            try:
                nuevo_nivel = float(input("Ingrese el nuevo nivel de yeso (debe ser 2m o menos): "))
                if nuevo_nivel <= 2:
                    return nuevo_nivel
                else:
                    print("El nivel debe ser 2m o menos.")
            except ValueError:
                print("Por favor, ingrese un número válido.")
    return nivel_actual

def validar_restricciones(molino, material):
    global alimentaciones_actuales, alimentaciones_en_progreso

    with lock:
        # Verificar si ya se está alimentando el mismo material al mismo molino
        if molino in alimentaciones_en_progreso and material in alimentaciones_en_progreso[molino]:
            return False, f"Ya se está alimentando {material} a {molino}."

        # Restricciones para Clinker
        if material == "Clinker":
            if any(alim[1] == "Clinker" for alim in alimentaciones_actuales):
                return False, "No se puede alimentar Clinker a múltiples molinos simultáneamente"

        # Restricciones para MC2
        if molino == "MC2":
            materiales_actuales = [alim[1] for alim in alimentaciones_actuales if alim[0] == "MC2"]
            if material in ["Puzolana_Humeda", "Puzolana_Seca"] and any(m in materiales_actuales for m in ["Puzolana_Humeda", "Puzolana_Seca"]):
                return False, "No se puede alimentar dos tipos de Puzolana simultáneamente a MC2"
            if material == "Yeso" and "Yeso" in materiales_actuales:
                return False, "Ya se está alimentando Yeso a MC2"

        # Restricciones para MC3
        if molino == "MC3":
            materiales_actuales = [alim[1] for alim in alimentaciones_actuales if alim[0] == "MC3"]
            if len(materiales_actuales) >= 2 and material not in materiales_actuales:
                return False, "No se pueden alimentar más de dos materiales a MC3 simultáneamente"
            if "Clinker" in materiales_actuales and material in ["Puzolana", "Yeso"]:
                return False, "No se puede alimentar Clinker y Puzolana/Yeso a MC3 simultáneamente"

        # Restricciones para MC1
        if molino == "MC1":
            if material in ["Puzolana", "Yeso"] and "Puzolana" in alimentaciones_en_progreso.get(molino, []) and "Yeso" in alimentaciones_en_progreso.get(molino, []):
                return False, "No se puede alimentar Puzolana y Yeso simultáneamente a MC1"

        # Restricción general para Yeso en MC1
        if molino == "MC1" and material == "Yeso":
            if niveles_MC1["Yeso"] + 2 > tolvas_MC1["Yeso"]["max_metros"]:
                return False, "No se puede alimentar más Yeso a MC1, se excedería el límite de 2m"

    return True, "Validación exitosa"

def imprimir_progreso(molino, material, porcentaje):
    barra = f"[{'#' * int(porcentaje / 2)}{' ' * (50 - int(porcentaje / 2))}]"
    with lock_print:
        sys.stdout.write(f"\r{molino} - Alimentando {material}: {barra} {porcentaje:.1f}%")
        sys.stdout.flush()

def simular_alimentacion(molino, material, cantidad, duracion_real):
    tiempo_inicio = time.time()
    tiempo_simulado = 0
    while tiempo_simulado < 60:  # 1 hora simulada
        tiempo_actual = time.time() - tiempo_inicio
        tiempo_simulado = (tiempo_actual / duracion_real) * 60
        porcentaje = min(tiempo_simulado / 60 * 100, 100)
        imprimir_progreso(molino, material, porcentaje)
        time.sleep(0.1)

    with lock_print:
        print(f"\n{molino} - Alimentación de {material} completada.")

    # Actualizar niveles después de la alimentación
    with lock:
        if molino == "MC1":
            niveles_MC1[material] += cantidad
        elif molino == "MC2":
            niveles_MC2[material] += cantidad
        elif molino == "MC3":
            niveles_MC3[material] += cantidad

        if molino in alimentaciones_en_progreso:
            alimentaciones_en_progreso[molino].remove(material)
        if (molino, material) in alimentaciones_actuales:
            alimentaciones_actuales.remove((molino, material))

        print(f"\n{molino} - Alimentación de {material} completada. Ruta habilitada para nueva alimentación.")

    if not alimentaciones_en_progreso.get(molino):
        temporizadores_molinos[molino] = False

def iniciar_alimentacion(molino, material, cantidad):
    if molino not in alimentaciones_en_progreso:
        alimentaciones_en_progreso[molino] = set()
    alimentaciones_en_progreso[molino].add(material)
    temporizadores_molinos[molino] = True
    thread = threading.Thread(target=simular_alimentacion, args=(molino, material, cantidad, 30))
    thread.start()
    print(f"\nIniciando alimentación de {material} a {molino}")

def intentar_alimentar(molino, material, cantidad):
    valido, mensaje = validar_restricciones(molino, material)
    if not valido:
        print(f"Error: No se puede alimentar {material} a {molino}: {mensaje}")
        return 0

    with lock:
        alimentaciones_actuales.add((molino, material))
        if molino not in alimentaciones_en_progreso:
            alimentaciones_en_progreso[molino] = set()
        alimentaciones_en_progreso[molino].add(material)

    if molino == "MC1":
        niveles = niveles_MC1
        tolvas = tolvas_MC1
    elif molino == "MC2":
        niveles = niveles_MC2
        tolvas = tolvas_MC2
    elif molino == "MC3":
        niveles = niveles_MC3
        tolvas = tolvas_MC3

    if material in ["Puzolana", "Puzolana_Humeda", "Puzolana_Seca"]:
        ajustar_velocidad("puzolana", 1100, 1150)
    elif material == "Yeso":
        ajustar_velocidad("yeso", 850, 900)

    capacidad = tolvas[material]['capacidad']
    if 'max_metros' in tolvas[material]:
        max_nivel = tolvas[material]['max_metros']
        nivel_actual = niveles[material] / capacidad * max_nivel
        if molino == "MC1" and material == "Yeso":
            max_nivel = min(max_nivel, 2)  # Limitar a 2m para yeso en MC1
        espacio_disponible = max_nivel - nivel_actual
        cantidad_metros = min(cantidad / capacidad * max_nivel, espacio_disponible)
        cantidad_alimentada = cantidad_metros / max_nivel * capacidad
    else:  # Para MC3 que usa porcentajes
        max_nivel = tolvas[material]['max_porcentaje']
        nivel_actual = niveles[material] / capacidad * 100
        espacio_disponible = max_nivel - nivel_actual
        cantidad_porcentaje = min(cantidad / capacidad * 100, espacio_disponible)
        cantidad_alimentada = cantidad_porcentaje / 100 * capacidad

    if cantidad_alimentada > 0:
        print(f"Iniciando alimentación de {cantidad_alimentada:.2f}t de {material} a {molino}")
        iniciar_alimentacion(molino, material, cantidad_alimentada)
    else:
        print(f"No se pudo alimentar {material} a {molino}. Tolva llena o se alcanzó el límite máximo.")
        with lock:
            alimentaciones_actuales.discard((molino, material))
            if molino in alimentaciones_en_progreso:
                alimentaciones_en_progreso[molino].discard(material)

    return cantidad_alimentada

def verificar_alimentacion_completa(molino, tipo_produccion, niveles):
    necesidades = calcular_necesidades(molino, tipo_produccion)
    materias_faltantes = []

    for material, cantidad_necesaria in necesidades.items():
        if cantidad_necesaria > 0 and niveles[material] == 0:
            materias_faltantes.append(material)

    return materias_faltantes

def todas_tolvas_llenas():
    rendimiento_MC1_MC2 = 0.8
    rendimiento_MC3 = 0.5

    if any(nivel < tolvas_MC1[material]['capacidad'] * rendimiento_MC1_MC2 for material, nivel in niveles_MC1.items()):
        return False
    if any(nivel < tolvas_MC2[material]['capacidad'] * rendimiento_MC1_MC2 for material, nivel in niveles_MC2.items()):
        return False
    if any(nivel < tolvas_MC3[material]['capacidad'] * rendimiento_MC3 for material, nivel in niveles_MC3.items()):
        return False
    return True

def calcular_necesidades(molino, tipo_produccion):
    produccion_MC1 = {
        "P30": (0.30, 0.015, 72),
        "P40": (0.40, 0.015, 64)
    }
    produccion_MC2 = {
        "P10": (0.10, 0.03, 70),
        "P16": (0.16, 0.025, 80),
        "P20": (0.12, 0.025, 87),
        "P30": (0.30, 0.02, 110)
    }
    produccion_MC3 = {
        "P30": (0.30, 0.025, 37)
    }

    if molino == "MC1" and tipo_produccion in produccion_MC1:
        puzolana, yeso, produccion = produccion_MC1[tipo_produccion]
        clinker = 1 - puzolana - yeso
        return {
            "Clinker": clinker * produccion,
            "Puzolana": puzolana * produccion,
            "Yeso": yeso * produccion
        }
    elif molino == "MC2" and tipo_produccion in produccion_MC2:
        puzolana, yeso, produccion = produccion_MC2[tipo_produccion]
        clinker = 1 - puzolana - yeso
        return {
            "Clinker": clinker * produccion,
            "Puzolana_Humeda": puzolana * produccion * 0.7,
            "Puzolana_Seca": puzolana * produccion * 0.3,
            "Yeso": yeso * produccion
        }
    elif molino == "MC3" and tipo_produccion in produccion_MC3:
        puzolana, yeso, produccion = produccion_MC3[tipo_produccion]
        clinker = 1 - puzolana - yeso
        return {
            "Clinker": clinker * produccion,
            "Puzolana": puzolana * produccion,
            "Yeso": yeso * produccion
        }

    print(f"Advertencia: No se encontró configuración para {molino} con producto {tipo_produccion}")
    return {"Clinker": 60, "Puzolana": 30, "Yeso": 10}

def redondear_diccionario(d, decimales=2):
    return {k: round(v, decimales) if isinstance(v, (int, float)) else v for k, v in d.items()}

def calcular_tiempos_llenado(molino, necesidades):
    tiempos_llenado = {}
    if molino == "MC1":
        rendimiento = 0.8
        for mat, cant in necesidades.items():
            if mat == "Yeso":
                capacidad_objetivo = min(tolvas_MC1[mat]["capacidad"] * rendimiento, 2 / tolvas_MC1[mat]["max_metros"] * tolvas_MC1[mat]["capacidad"])
            else:
                capacidad_objetivo = tolvas_MC1[mat]["capacidad"] * rendimiento
            nivel_actual = niveles_MC1[mat]
            if cant > 0:
                tiempos_llenado[mat] = (capacidad_objetivo - nivel_actual) / cant
            else:
                tiempos_llenado[mat] = float('inf')

    elif molino == "MC2":
        rendimiento = 0.8
        for mat, cant in necesidades.items():
            capacidad_objetivo = tolvas_MC2[mat]["capacidad"] * rendimiento
            nivel_actual = niveles_MC2[mat]
            if cant > 0:
                tiempos_llenado[mat] = (capacidad_objetivo - nivel_actual) / cant
            else:
                tiempos_llenado[mat] = float('inf')
    else:  # MC3
        rendimiento = 0.5
        for mat, cant in necesidades.items():
            capacidad_objetivo = tolvas_MC3[mat]["capacidad"] * rendimiento
            nivel_actual = niveles_MC3[mat]
            if cant > 0:
                tiempos_llenado[mat] = (capacidad_objetivo - nivel_actual) / cant
            else:
                tiempos_llenado[mat] = float('inf')

    return tiempos_llenado

def calcular_tiempo_vaciado(molino, material):
    if molino == "MC1":
        niveles = niveles_MC1
        tolvas = tolvas_MC1
    elif molino == "MC2":
        niveles = niveles_MC2
        tolvas = tolvas_MC2
    else:  # MC3
        niveles = niveles_MC3
        tolvas = tolvas_MC3

    # Manejar el caso especial de Puzolana para MC1 y MC3
    if material == "Puzolana" and molino in ["MC1", "MC3"]:
        capacidad = tolvas["Puzolana"]['capacidad']
        nivel_actual = niveles["Puzolana"]
    elif material == "Puzolana_Humeda" and molino == "MC2":
        capacidad = tolvas["Puzolana_Humeda"]['capacidad']
        nivel_actual = niveles["Puzolana_Humeda"]
    elif material == "Puzolana_Seca" and molino == "MC2":
        capacidad = tolvas["Puzolana_Seca"]['capacidad']
        nivel_actual = niveles["Puzolana_Seca"]
    else:
        capacidad = tolvas[material]['capacidad']
        nivel_actual = niveles[material]

    consumo = calcular_consumo(molino, material)

    if consumo > 0:
        return (nivel_actual - nivel_ineficiente(molino, material)) / consumo
    else:
        return float('inf')

def nivel_ineficiente(molino, material):
    if molino in ["MC1", "MC2"]:
        if molino == "MC1":
            return 0.2 * tolvas_MC1[material]['capacidad']  # 20% de la capacidad
        else:  # MC2
            if material in ["Puzolana_Humeda", "Puzolana_Seca"]:
                return 0.2 * tolvas_MC2[material]['capacidad']  # 20% de la capacidad
            else:
                return 0.2 * tolvas_MC2[material]['capacidad']  # 20% de la capacidad
    else:  # MC3
        return 0.5 * tolvas_MC3[material]['capacidad']  # 50% de la capacidad

def calcular_consumo(molino, material):
    tipo_produccion = obtener_tipo_produccion_actual(molino)
    necesidades = calcular_necesidades(molino, tipo_produccion)
    if molino == "MC2" and material in ["Puzolana_Humeda", "Puzolana_Seca"]:
        return necesidades.get("Puzolana_Humeda", 0) + necesidades.get("Puzolana_Seca", 0)
    return necesidades.get(material, 0)

def obtener_tipo_produccion_actual(molino):
    # Esta función debería obtener el tipo de producción actual del molino
    # Por ahora, usaremos un valor predeterminado
    if molino == "MC1":
        return "P30"
    elif molino == "MC2":
        return "P16"
    else:  # MC3
        return "P30"

def seleccionar_ruta_alimentacion(molino, material):
    if molino == "MC1":
        if material == "Clinker":
            return "Hacia MC1 desde Pretrit"
        elif material == "Puzolana":
            tiempo_vaciado_MC1 = calcular_tiempo_vaciado("MC1", "Puzolana")
            tiempo_vaciado_MC2 = calcular_tiempo_vaciado("MC2", "Puzolana_Humeda")
            return "Húmeda a MC1 por MC1" if tiempo_vaciado_MC1 < tiempo_vaciado_MC2 else "Húmeda a MC1 por MC2"
        elif material == "Yeso":
            tiempo_vaciado_MC1 = calcular_tiempo_vaciado("MC1", "Yeso")
            tiempo_vaciado_MC2 = calcular_tiempo_vaciado("MC2", "Yeso")
            return "Hacia MC1 por MC1" if tiempo_vaciado_MC1 < tiempo_vaciado_MC2 else "Hacia MC1 por MC2"
    elif molino == "MC2":
        if material == "Clinker":
            return "Hacia MC2 desde Pretrit"
        elif material == "Puzolana_Humeda":
            return "P.H. a 426HO04 por MC2"
        elif material == "Puzolana_Seca":
            return "P.S a 426HO02 por 426HO04"
        elif material == "Yeso":
            return "Hacia MC2 por MC2"
    elif molino == "MC3":
        if material == "Clinker":
            if niveles_MC3["Clinker_Silo_Blanco"] / tolvas_MC3["Clinker_Silo_Blanco"]["capacidad"] > 0.3:
                return "Hacia MC3 desde Silo3"
            else:
                return "Hacia Silo 3 desde Pretrit"
        elif material == "Puzolana":
            return "P. seca a MC3 por MC2"
        elif material == "Yeso":
            tiempo_vaciado_MC1 = calcular_tiempo_vaciado("MC1", "Yeso")
            tiempo_vaciado_MC2 = calcular_tiempo_vaciado("MC2", "Yeso")
            tiempo_vaciado_MC3 = calcular_tiempo_vaciado("MC3", "Yeso")
            if tiempo_vaciado_MC1 <= tiempo_vaciado_MC2 and tiempo_vaciado_MC1 <= tiempo_vaciado_MC3:
                return "Hacia MC3 por MC1"
            elif tiempo_vaciado_MC2 <= tiempo_vaciado_MC1 and tiempo_vaciado_MC2 <= tiempo_vaciado_MC3:
                return "Hacia MC3 por MC2"
            else:
                return "Hacia MC3 por MC3"
    return "Ruta no especificada"

def imprimir_tiempos(molino, tipo_produccion):
    necesidades = calcular_necesidades(molino, tipo_produccion)
    tiempos_llenado = calcular_tiempos_llenado(molino, necesidades)

    print(f"\nTiempos de llenado y vaciado para {molino} - {tipo_produccion}:")
    print("-" * 60)
    print(f"{'Material':<15}{'Tiempo Llenado (h)':<20}{'Tiempo Vaciado (h)':<20}")
    print("-" * 60)

    for material, tiempo_llenado in tiempos_llenado.items():
        tiempo_vaciado = calcular_tiempo_vaciado(molino, material)
        tiempo_llenado_str = f"{tiempo_llenado:.2f}" if tiempo_llenado != float('inf') else "∞"
        tiempo_vaciado_str = f"{tiempo_vaciado:.2f}" if tiempo_vaciado != float('inf') else "∞"
        print(f"{material:<15}{tiempo_llenado_str:<20}{tiempo_vaciado_str:<20}")

def generar_recomendaciones(molino_actual):
    recomendaciones = []
    restricciones = []

    def espacio_disponible(nivel_actual, capacidad, rendimiento):
        return capacidad * rendimiento - nivel_actual

    if molino_actual == "MC1":
        rendimiento = 0.8
        esp_clinker = espacio_disponible(niveles_MC1["Clinker"], tolvas_MC1["Clinker"]["capacidad"], rendimiento)
        esp_puzolana = espacio_disponible(niveles_MC1["Puzolana"], tolvas_MC1["Puzolana"]["capacidad"], rendimiento)
        esp_yeso = min(espacio_disponible(niveles_MC1["Yeso"], tolvas_MC1["Yeso"]["capacidad"], rendimiento), 60 - niveles_MC1["Yeso"])

        if esp_clinker > 0:
            recomendaciones.append(f"Recomiendo alimentar Clinker a MC1 desde Pretrit (Espacio disponible: {esp_clinker:.2f}t)")
        if esp_puzolana > 0:
            recomendaciones.append(f"Recomiendo alimentar Puzolana a MC1 (Espacio disponible: {esp_puzolana:.2f}t)")
        if esp_yeso > 0:
            recomendaciones.append(f"Recomiendo alimentar Yeso a MC1 (Espacio disponible: {esp_yeso:.2f}t)")

    elif molino_actual == "MC2":
        rendimiento = 0.8
        esp_clinker = espacio_disponible(niveles_MC2["Clinker"], tolvas_MC2["Clinker"]["capacidad"], rendimiento)
        esp_puzolana_h = espacio_disponible(niveles_MC2["Puzolana_Humeda"], tolvas_MC2["Puzolana_Humeda"]["capacidad"], rendimiento)
        esp_puzolana_s = espacio_disponible(niveles_MC2["Puzolana_Seca"], tolvas_MC2["Puzolana_Seca"]["capacidad"], rendimiento)
        esp_yeso = espacio_disponible(niveles_MC2["Yeso"], tolvas_MC2["Yeso"]["capacidad"], rendimiento)

        if esp_clinker > 0:
            recomendaciones.append(f"Recomiendo alimentar Clinker a MC2 desde Pretrit (Espacio disponible: {esp_clinker:.2f}t)")
        if esp_puzolana_h > 0:
            recomendaciones.append(f"Recomiendo alimentar Puzolana Húmeda a 426HO04 por MC2 (Espacio disponible: {esp_puzolana_h:.2f}t)")
        if esp_puzolana_s > 0:
            recomendaciones.append(f"Recomiendo alimentar Puzolana Seca a 426HO02 por 426HO04 (Espacio disponible: {esp_puzolana_s:.2f}t)")
        if esp_yeso > 0:
            recomendaciones.append(f"Recomiendo alimentar Yeso a MC2 por MC2 (Espacio disponible: {esp_yeso:.2f}t)")

    elif molino_actual == "MC3":
        rendimiento = 0.5
        esp_clinker = espacio_disponible(niveles_MC3["Clinker"], tolvas_MC3["Clinker"]["capacidad"], rendimiento)
        esp_clinker_sb = espacio_disponible(niveles_MC3["Clinker_Silo_Blanco"], tolvas_MC3["Clinker_Silo_Blanco"]["capacidad"], rendimiento)
        esp_puzolana = espacio_disponible(niveles_MC3["Puzolana"], tolvas_MC3["Puzolana"]["capacidad"], rendimiento)
        esp_yeso = espacio_disponible(niveles_MC3["Yeso"], tolvas_MC3["Yeso"]["capacidad"], rendimiento)

        if esp_clinker > 0:
            if niveles_MC3["Clinker_Silo_Blanco"] / tolvas_MC3["Clinker_Silo_Blanco"]["capacidad"] > 0.3:
                recomendaciones.append(f"Recomiendo alimentar Clinker a MC3 desde Silo Blanco (Espacio disponible: {esp_clinker:.2f}t)")
            else:
                recomendaciones.append(f"Recomiendo alimentar Clinker a Silo Blanco desde Pretrit (Espacio disponible: {esp_clinker_sb:.2f}t)")
        if esp_puzolana > 0:
            recomendaciones.append(f"Recomiendo alimentar Puzolana Seca a MC3 por MC2 (Espacio disponible: {esp_puzolana:.2f}t)")
        if esp_yeso > 0:
            recomendaciones.append(f"Recomiendo alimentar Yeso a MC3 (Espacio disponible: {esp_yeso:.2f}t)")

    if not recomendaciones:
        recomendaciones.append(f"Todas las tolvas de {molino_actual} están llenas o cerca de su capacidad máxima.")

    restricciones.append("Es necesario vaciar el sistema antes de una nueva alimentación para evitar contaminación.")
    restricciones.append("No se puede alimentar simultáneamente Clinker, Puzolana y Yeso a MC3.")
    restricciones.append("No se puede alimentar Clinker a múltiples molinos simultáneamente.")

    return recomendaciones, restricciones

# def optimizar_alimentacion(molino, tipo_produccion):
#     necesidades = calcular_necesidades(molino, tipo_produccion)
#     print(f"\nNecesidades para {molino} - {tipo_produccion}:")
#     for material, cantidad in necesidades.items():
#         print(f"{material}: {cantidad:.2f} t/h")

#     imprimir_tiempos(molino, tipo_produccion)

#     recomendaciones, restricciones = generar_recomendaciones(molino)
#     print("\nRecomendaciones:")
#     for recomendacion in recomendaciones:
#         print(recomendacion)

#     materiales_alimentados = []
#     for material, cantidad_necesaria in necesidades.items():
#         if cantidad_necesaria > 0:
#             ruta = seleccionar_ruta_alimentacion(molino, material)
#             print(f"\nIntentando alimentar {material} a {molino} por la ruta: {ruta}")
#             cantidad_alimentada = intentar_alimentar(molino, material, cantidad_necesaria)
#             if cantidad_alimentada > 0:
#                 materiales_alimentados.append(material)

#     print("\nRestricciones aplicables:")
#     restricciones_aplicables = [r for r in restricciones if any(mat in r for mat in materiales_alimentados)]
#     for restriccion in restricciones_aplicables:
#         print(restriccion)

#     print("\nTiempos actualizados después de la alimentación:")
#     imprimir_tiempos(molino, tipo_produccion)

# def main():
#     global niveles_MC1, niveles_MC2, niveles_MC3
#     ciclo = 1

#     while True:
#         print(f"\n--- Ciclo de optimización {ciclo} ---")

#         for molino in ["MC1", "MC2", "MC3"]:
#             if molino == "MC1":
#                 tipos_validos = ["P30", "P40"]
#             elif molino == "MC2":
#                 tipos_validos = ["P10", "P16", "P20", "P30"]
#             elif molino == "MC3":
#                 tipos_validos = ["P30"]

#             print(f"\nOptimizando {molino}")
#             print(f"Este molino puede producir: {', '.join(tipos_validos)}")
#             tipo_produccion = input(f"Ingrese el tipo de producción para {molino} (o presione Enter para omitir): ").upper()

#             if tipo_produccion:
#                 if tipo_produccion not in tipos_validos:
#                     print(f"Error, Ingrese un tipo de producción válido para {molino}")
#                     continue
#                 optimizar_alimentacion(molino, tipo_produccion)

#         print("\nEstado actual de las alimentaciones:")
#         for molino, materiales in alimentaciones_en_progreso.items():
#             if materiales:
#                 print(f"{molino}: {', '.join(materiales)}")
#             else:
#                 print(f"{molino}: Sin alimentaciones en progreso")

#         # Esperar a que todos los temporizadores terminen
#         while any(temporizadores_molinos.values()):
#             time.sleep(1)

#         print("\nTodas las alimentaciones han sido completadas.")
#         print("\nEstado actual después de la alimentación:")
#         print(f"Niveles MC1: {redondear_diccionario(niveles_MC1)}")
#         print(f"Niveles MC2: {redondear_diccionario(niveles_MC2)}")
#         print(f"Niveles MC3: {redondear_diccionario(niveles_MC3)}")

#         if todas_tolvas_llenas():
#             print("Todas las tolvas están llenas. Finalizando la simulación.")
#             break

#         ciclo += 1

#         continuar = input("¿Desea continuar con otro ciclo? (s/n): ").lower()
#         if continuar != 's':
#             break

#     print("Simulación completada.")

# if __name__ == "__main__":
#     main()

def verificar_alimentacion_completa(molino, tipo_produccion, niveles):
    necesidades = calcular_necesidades(molino, tipo_produccion)
    materias_faltantes = []

    for material, cantidad_necesaria in necesidades.items():
        if cantidad_necesaria > 0 and niveles[material] == 0:
            materias_faltantes.append(material)

    return materias_faltantes

def optimizar_alimentacion(molino, tipo_produccion):
    global niveles_MC1, niveles_MC2, niveles_MC3
    necesidades = calcular_necesidades(molino, tipo_produccion)
    print(f"\nNecesidades para {molino} - {tipo_produccion}:")
    for material, cantidad in necesidades.items():
        print(f"{material}: {cantidad:.2f} t/h")

    imprimir_tiempos(molino, tipo_produccion)

    recomendaciones, restricciones = generar_recomendaciones(molino)
    print("\nRecomendaciones:")
    for recomendacion in recomendaciones:
        print(recomendacion)

    materiales_alimentados = []
    for material, cantidad_necesaria in necesidades.items():
        if cantidad_necesaria > 0:
            ruta = seleccionar_ruta_alimentacion(molino, material)
            print(f"\nIntentando alimentar {material} a {molino} por la ruta: {ruta}")
            cantidad_alimentada = intentar_alimentar(molino, material, cantidad_necesaria)
            if cantidad_alimentada > 0:
                materiales_alimentados.append(material)

    print("\nRestricciones aplicables:")
    restricciones_aplicables = [r for r in restricciones if any(mat in r for mat in materiales_alimentados)]
    for restriccion in restricciones_aplicables:
        print(restriccion)

    print("\nTiempos actualizados después de la alimentación:")
    imprimir_tiempos(molino, tipo_produccion)

    # Verificar si todas las materias primas necesarias fueron alimentadas
    if molino == "MC1":
        niveles = niveles_MC1
    elif molino == "MC2":
        niveles = niveles_MC2
    else:  # MC3
        niveles = niveles_MC3

    materias_faltantes = verificar_alimentacion_completa(molino, tipo_produccion, niveles)
    if materias_faltantes:
        print(f"\nAdvertencia: No se pudo alimentar todas las materias primas necesarias para {molino}.")
        print(f"Materias primas faltantes: {', '.join(materias_faltantes)}")
        print("No se puede elaborar el producto con la configuración actual.")
    else:
        print(f"\nTodas las materias primas necesarias para {molino} han sido alimentadas correctamente.")
        print("El producto puede ser elaborado con la configuración actual.")

def main():
    global niveles_MC1, niveles_MC2, niveles_MC3
    ciclo = 1

    while True:
        print(f"\n--- Ciclo de optimización {ciclo} ---")

        for molino in ["MC1", "MC2", "MC3"]:
            if molino == "MC1":
                tipos_validos = ["P30", "P40"]
            elif molino == "MC2":
                tipos_validos = ["P10", "P16", "P20", "P30"]
            elif molino == "MC3":
                tipos_validos = ["P30"]

            print(f"\nOptimizando {molino}")
            print(f"Este molino puede producir: {', '.join(tipos_validos)}")
            tipo_produccion = input(f"Ingrese el tipo de producción para {molino} (o presione Enter para omitir): ").upper()

            if tipo_produccion:
                if tipo_produccion not in tipos_validos:
                    print(f"Error, Ingrese un tipo de producción válido para {molino}")
                    continue
                optimizar_alimentacion(molino, tipo_produccion)

        print("\nEstado actual de las alimentaciones:")
        for molino, materiales in alimentaciones_en_progreso.items():
            if materiales:
                print(f"{molino}: {', '.join(materiales)}")
            else:
                print(f"{molino}: Sin alimentaciones en progreso")

        # Esperar a que todos los temporizadores terminen
        while any(temporizadores_molinos.values()):
            time.sleep(1)

        print("\nTodas las alimentaciones en progreso han sido completadas.")
        print("\nEstado actual después de la alimentación:")
        print(f"Niveles MC1: {redondear_diccionario(niveles_MC1)}")
        print(f"Niveles MC2: {redondear_diccionario(niveles_MC2)}")
        print(f"Niveles MC3: {redondear_diccionario(niveles_MC3)}")

        if todas_tolvas_llenas():
            print("Todas las tolvas están llenas. Finalizando la simulación.")
            break

        ciclo += 1

        continuar = input("¿Desea continuar con otro ciclo? (s/n): ").lower()
        if continuar != 's':
            break

    print("Simulación completada.")

if __name__ == "__main__":
    main()


--- Ciclo de optimización 1 ---

Optimizando MC1
Este molino puede producir: P30, P40
Ingrese el tipo de producción para MC1 (o presione Enter para omitir): P30

Necesidades para MC1 - P30:
Clinker: 49.32 t/h
Puzolana: 21.60 t/h
Yeso: 1.08 t/h

Tiempos de llenado y vaciado para MC1 - P30:
------------------------------------------------------------
Material       Tiempo Llenado (h)  Tiempo Vaciado (h)  
------------------------------------------------------------
Clinker        8.11                -2.03               
Puzolana       11.11               -2.78               
Yeso           55.56               -55.56              

Recomendaciones:
Recomiendo alimentar Clinker a MC1 desde Pretrit (Espacio disponible: 400.00t)
Recomiendo alimentar Puzolana a MC1 (Espacio disponible: 240.00t)
Recomiendo alimentar Yeso a MC1 (Espacio disponible: 60.00t)

Intentando alimentar Clinker a MC1 por la ruta: Hacia MC1 desde Pretrit
Iniciando alimentación de 49.32t de Clinker a MC1
MC1 - Alimentand

KeyboardInterrupt: Interrupted by user

In [ ]:
# codigo bien ; fecha:11/09/24
import random
import time
import threading
import sys

# Definición de capacidades de tolvas y niveles iniciales
tolvas_MC1 = {
    "Clinker": {"capacidad": 500, "max_metros": 14},
    "Puzolana": {"capacidad": 300, "max_metros": 12},
    "Yeso": {"capacidad": 300, "max_metros": 10}
}
tolvas_MC2 = {
    "Clinker": {"capacidad": 300, "max_metros": 9},
    "Puzolana_Humeda": {"capacidad": 500, "max_metros": 15, "tolva": "426HO04"},
    "Puzolana_Seca": {"capacidad": 100, "max_metros": 12, "tolva": "426HO02"},
    "Yeso": {"capacidad": 120, "max_metros": 9}
}
tolvas_MC3 = {
    "Clinker": {"capacidad": 60, "max_porcentaje": 100},
    "Clinker_Silo_Blanco": {"capacidad": 500, "max_metros": 10.5},
    "Puzolana": {"capacidad": 35, "max_porcentaje": 100},
    "Yeso": {"capacidad": 30, "max_porcentaje": 100}
}

niveles_MC1 = {"Clinker": 0, "Puzolana": 0, "Yeso": 0}
niveles_MC2 = {"Clinker": 0, "Puzolana_Humeda": 0, "Puzolana_Seca": 0, "Yeso": 0}
niveles_MC3 = {"Clinker": 0, "Clinker_Silo_Blanco": 0, "Puzolana": 0, "Yeso": 0}

# Variables globales para rastrear alimentaciones y temporizadores
alimentaciones_actuales = set()
alimentaciones_en_progreso = {}
temporizadores_molinos = {}
lock = threading.Lock()
lock_print = threading.Lock()

def resetear_niveles_molino(molino):
    global niveles_MC1, niveles_MC2, niveles_MC3
    if molino == "MC1":
        niveles_MC1 = {"Clinker": 0, "Puzolana": 0, "Yeso": 0}
    elif molino == "MC2":
        niveles_MC2 = {"Clinker": 0, "Puzolana_Humeda": 0, "Puzolana_Seca": 0, "Yeso": 0}
    elif molino == "MC3":
        niveles_MC3 = {"Clinker": 0, "Clinker_Silo_Blanco": 0, "Puzolana": 0, "Yeso": 0}

def ajustar_velocidad(material, min_vel, max_vel):
    while True:
        try:
            velocidad = float(input(f" Ingrese la velocidad del transportador para {material} ({min_vel}-{max_vel} RPM): "))
            if min_vel <= velocidad <= max_vel:
                print(f" Ajustando velocidad del transportador a {velocidad:.2f} RPM para {material}")
                return velocidad
            else:
                print(f"La velocidad debe estar entre {min_vel} y {max_vel} RPM.")
        except ValueError:
            print("Por favor, ingrese un número válido.")

def verificar_nivel_yeso(molino, nivel_actual):
    if molino == "MC1" and nivel_actual > 2:
        print(f"Advertencia: El nivel de yeso en {molino} ({nivel_actual:.2f}m) excede el límite de 2m.")
        while True:
            try:
                nuevo_nivel = float(input("Ingrese el nuevo nivel de yeso (debe ser 2m o menos): "))
                if nuevo_nivel <= 2:
                    return nuevo_nivel
                else:
                    print("El nivel debe ser 2m o menos.")
            except ValueError:
                print("Por favor, ingrese un número válido.")
    return nivel_actual

def validar_restricciones(molino, material):
    global alimentaciones_actuales, alimentaciones_en_progreso

    with lock:
        # Verificar si ya se está alimentando el mismo material al mismo molino
        if molino in alimentaciones_en_progreso and material in alimentaciones_en_progreso[molino]:
            return False, f"Ya se está alimentando {material} a {molino}."

        # Restricciones para Clinker
        if material == "Clinker":
            if any(alim[1] == "Clinker" for alim in alimentaciones_actuales):
                return False, "No se puede alimentar Clinker a múltiples molinos simultáneamente"

        # Restricciones para MC2
        if molino == "MC2":
            materiales_actuales = [alim[1] for alim in alimentaciones_actuales if alim[0] == "MC2"]
            if material in ["Puzolana_Humeda", "Puzolana_Seca"] and any(m in materiales_actuales for m in ["Puzolana_Humeda", "Puzolana_Seca"]):
                return False, "No se puede alimentar dos tipos de Puzolana simultáneamente a MC2"
            if material == "Yeso" and "Yeso" in materiales_actuales:
                return False, "Ya se está alimentando Yeso a MC2"

        # Restricciones para MC3
        if molino == "MC3":
            materiales_actuales = [alim[1] for alim in alimentaciones_actuales if alim[0] == "MC3"]
            if len(materiales_actuales) >= 2 and material not in materiales_actuales:
                return False, "No se pueden alimentar más de dos materiales a MC3 simultáneamente"
            if "Clinker" in materiales_actuales and material in ["Puzolana", "Yeso"]:
                return False, "No se puede alimentar Clinker y Puzolana/Yeso a MC3 simultáneamente"

        # Restricciones para MC1
        if molino == "MC1":
            if material in ["Puzolana", "Yeso"] and "Puzolana" in alimentaciones_en_progreso.get(molino, []) and "Yeso" in alimentaciones_en_progreso.get(molino, []):
                return False, "No se puede alimentar Puzolana y Yeso simultáneamente a MC1"

        # Restricción general para Yeso en MC1
        if molino == "MC1" and material == "Yeso":
            if niveles_MC1["Yeso"] + 2 > tolvas_MC1["Yeso"]["max_metros"]:
                return False, "No se puede alimentar más Yeso a MC1, se excedería el límite de 2m"

    return True, "Validación exitosa"

def imprimir_progreso(molino, material, porcentaje):
    barra = f"[{'#' * int(porcentaje / 2)}{' ' * (50 - int(porcentaje / 2))}]"
    with lock_print:
        sys.stdout.write(f"\r{molino} - Alimentando {material}: {barra} {porcentaje:.1f}%")
        sys.stdout.flush()

def simular_alimentacion(molino, material, cantidad, duracion_real):
    tiempo_inicio = time.time()
    tiempo_simulado = 0
    while tiempo_simulado < 60:  # 1 hora simulada
        tiempo_actual = time.time() - tiempo_inicio
        tiempo_simulado = (tiempo_actual / duracion_real) * 60
        porcentaje = min(tiempo_simulado / 60 * 100, 100)
        imprimir_progreso(molino, material, porcentaje)
        time.sleep(0.1)

    with lock_print:
        print(f"\n{molino} - Alimentación de {material} completada.")

    # Actualizar niveles después de la alimentación
    with lock:
        if molino == "MC1":
            niveles_MC1[material] += cantidad
        elif molino == "MC2":
            niveles_MC2[material] += cantidad
        elif molino == "MC3":
            niveles_MC3[material] += cantidad

        if molino in alimentaciones_en_progreso:
            alimentaciones_en_progreso[molino].remove(material)
        if (molino, material) in alimentaciones_actuales:
            alimentaciones_actuales.remove((molino, material))

        print(f"\n{molino} - Alimentación de {material} completada. Ruta habilitada para nueva alimentación.")

    if not alimentaciones_en_progreso.get(molino):
        temporizadores_molinos[molino] = False

def iniciar_alimentacion(molino, material, cantidad):
    if molino not in alimentaciones_en_progreso:
        alimentaciones_en_progreso[molino] = set()
    alimentaciones_en_progreso[molino].add(material)
    temporizadores_molinos[molino] = True
    thread = threading.Thread(target=simular_alimentacion, args=(molino, material, cantidad, 30))
    thread.start()
    print(f"\nIniciando alimentación de {material} a {molino}")

def intentar_alimentar(molino, material, cantidad):
    valido, mensaje = validar_restricciones(molino, material)
    if not valido:
        print(f"Error: No se puede alimentar {material} a {molino}: {mensaje}")
        return 0

    with lock:
        alimentaciones_actuales.add((molino, material))
        if molino not in alimentaciones_en_progreso:
            alimentaciones_en_progreso[molino] = set()
        alimentaciones_en_progreso[molino].add(material)

    if molino == "MC1":
        niveles = niveles_MC1
        tolvas = tolvas_MC1
    elif molino == "MC2":
        niveles = niveles_MC2
        tolvas = tolvas_MC2
    elif molino == "MC3":
        niveles = niveles_MC3
        tolvas = tolvas_MC3

    if material in ["Puzolana", "Puzolana_Humeda", "Puzolana_Seca"]:
        ajustar_velocidad("puzolana", 1100, 1150)
    elif material == "Yeso":
        ajustar_velocidad("yeso", 850, 900)

    capacidad = tolvas[material]['capacidad']
    if 'max_metros' in tolvas[material]:
        max_nivel = tolvas[material]['max_metros']
        nivel_actual = niveles[material] / capacidad * max_nivel
        if molino == "MC1" and material == "Yeso":
            max_nivel = min(max_nivel, 2)  # Limitar a 2m para yeso en MC1
        espacio_disponible = max_nivel - nivel_actual
        cantidad_metros = min(cantidad / capacidad * max_nivel, espacio_disponible)
        cantidad_alimentada = cantidad_metros / max_nivel * capacidad
    else:  # Para MC3 que usa porcentajes
        max_nivel = tolvas[material]['max_porcentaje']
        nivel_actual = niveles[material] / capacidad * 100
        espacio_disponible = max_nivel - nivel_actual
        cantidad_porcentaje = min(cantidad / capacidad * 100, espacio_disponible)
        cantidad_alimentada = cantidad_porcentaje / 100 * capacidad

    if cantidad_alimentada > 0:
        print(f"Iniciando alimentación de {cantidad_alimentada:.2f}t de {material} a {molino}")
        iniciar_alimentacion(molino, material, cantidad_alimentada)
    else:
        print(f"No se pudo alimentar {material} a {molino}. Tolva llena o se alcanzó el límite máximo.")
        with lock:
            alimentaciones_actuales.discard((molino, material))
            if molino in alimentaciones_en_progreso:
                alimentaciones_en_progreso[molino].discard(material)

    return cantidad_alimentada

def verificar_alimentacion_completa(molino, tipo_produccion, niveles):
    necesidades = calcular_necesidades(molino, tipo_produccion)
    materias_faltantes = []

    for material, cantidad_necesaria in necesidades.items():
        if cantidad_necesaria > 0 and niveles[material] == 0:
            materias_faltantes.append(material)

    return materias_faltantes

def todas_tolvas_llenas():
    rendimiento_MC1_MC2 = 0.8
    rendimiento_MC3 = 0.5

    if any(nivel < tolvas_MC1[material]['capacidad'] * rendimiento_MC1_MC2 for material, nivel in niveles_MC1.items()):
        return False
    if any(nivel < tolvas_MC2[material]['capacidad'] * rendimiento_MC1_MC2 for material, nivel in niveles_MC2.items()):
        return False
    if any(nivel < tolvas_MC3[material]['capacidad'] * rendimiento_MC3 for material, nivel in niveles_MC3.items()):
        return False
    return True

def calcular_necesidades(molino, tipo_produccion):
    produccion_MC1 = {
        "P30": (0.30, 0.015, 72),
        "P40": (0.40, 0.015, 64)
    }
    produccion_MC2 = {
        "P10": (0.10, 0.03, 70),
        "P16": (0.16, 0.025, 80),
        "P20": (0.12, 0.025, 87),
        "P30": (0.30, 0.02, 110)
    }
    produccion_MC3 = {
        "P30": (0.30, 0.025, 37)
    }

    if molino == "MC1" and tipo_produccion in produccion_MC1:
        puzolana, yeso, produccion = produccion_MC1[tipo_produccion]
        clinker = 1 - puzolana - yeso
        return {
            "Clinker": clinker * produccion,
            "Puzolana": puzolana * produccion,
            "Yeso": yeso * produccion
        }
    elif molino == "MC2" and tipo_produccion in produccion_MC2:
        puzolana, yeso, produccion = produccion_MC2[tipo_produccion]
        clinker = 1 - puzolana - yeso
        return {
            "Clinker": clinker * produccion,
            "Puzolana_Humeda": puzolana * produccion * 0.7,
            "Puzolana_Seca": puzolana * produccion * 0.3,
            "Yeso": yeso * produccion
        }
    elif molino == "MC3" and tipo_produccion in produccion_MC3:
        puzolana, yeso, produccion = produccion_MC3[tipo_produccion]
        clinker = 1 - puzolana - yeso
        return {
            "Clinker": clinker * produccion,
            "Puzolana": puzolana * produccion,
            "Yeso": yeso * produccion
        }

    print(f"Advertencia: No se encontró configuración para {molino} con producto {tipo_produccion}")
    return {"Clinker": 60, "Puzolana": 30, "Yeso": 10}

def redondear_diccionario(d, decimales=2):
    return {k: round(v, decimales) if isinstance(v, (int, float)) else v for k, v in d.items()}

def calcular_tiempos_llenado(molino, necesidades):
    tiempos_llenado = {}
    if molino == "MC1":
        rendimiento = 0.8
        for mat, cant in necesidades.items():
            if mat == "Yeso":
                capacidad_objetivo = min(tolvas_MC1[mat]["capacidad"] * rendimiento, 2 / tolvas_MC1[mat]["max_metros"] * tolvas_MC1[mat]["capacidad"])
            else:
                capacidad_objetivo = tolvas_MC1[mat]["capacidad"] * rendimiento
            nivel_actual = niveles_MC1[mat]
            if cant > 0:
                tiempos_llenado[mat] = (capacidad_objetivo - nivel_actual) / cant
            else:
                tiempos_llenado[mat] = float('inf')

    elif molino == "MC2":
        rendimiento = 0.8
        for mat, cant in necesidades.items():
            capacidad_objetivo = tolvas_MC2[mat]["capacidad"] * rendimiento
            nivel_actual = niveles_MC2[mat]
            if cant > 0:
                tiempos_llenado[mat] = (capacidad_objetivo - nivel_actual) / cant
            else:
                tiempos_llenado[mat] = float('inf')
    else:  # MC3
        rendimiento = 0.5
        for mat, cant in necesidades.items():
            capacidad_objetivo = tolvas_MC3[mat]["capacidad"] * rendimiento
            nivel_actual = niveles_MC3[mat]
            if cant > 0:
                tiempos_llenado[mat] = (capacidad_objetivo - nivel_actual) / cant
            else:
                tiempos_llenado[mat] = float('inf')

    return tiempos_llenado

def calcular_tiempo_vaciado(molino, material):
    if molino == "MC1":
        niveles = niveles_MC1
        tolvas = tolvas_MC1
    elif molino == "MC2":
        niveles = niveles_MC2
        tolvas = tolvas_MC2
    else:  # MC3
        niveles = niveles_MC3
        tolvas = tolvas_MC3

    # Manejar el caso especial de Puzolana para MC1 y MC3
    if material == "Puzolana" and molino in ["MC1", "MC3"]:
        capacidad = tolvas["Puzolana"]['capacidad']
        nivel_actual = niveles["Puzolana"]
    elif material == "Puzolana_Humeda" and molino == "MC2":
        capacidad = tolvas["Puzolana_Humeda"]['capacidad']
        nivel_actual = niveles["Puzolana_Humeda"]
    elif material == "Puzolana_Seca" and molino == "MC2":
        capacidad = tolvas["Puzolana_Seca"]['capacidad']
        nivel_actual = niveles["Puzolana_Seca"]
    else:
        capacidad = tolvas[material]['capacidad']
        nivel_actual = niveles[material]

    consumo = calcular_consumo(molino, material)

    if consumo > 0:
        return (nivel_actual - nivel_ineficiente(molino, material)) / consumo
    else:
        return float('inf')

def nivel_ineficiente(molino, material):
    if molino in ["MC1", "MC2"]:
        if molino == "MC1":
            return 0.2 * tolvas_MC1[material]['capacidad']  # 20% de la capacidad
        else:  # MC2
            if material in ["Puzolana_Humeda", "Puzolana_Seca"]:
                return 0.2 * tolvas_MC2[material]['capacidad']  # 20% de la capacidad
            else:
                return 0.2 * tolvas_MC2[material]['capacidad']  # 20% de la capacidad
    else:  # MC3
        return 0.5 * tolvas_MC3[material]['capacidad']  # 50% de la capacidad

def calcular_consumo(molino, material):
    tipo_produccion = obtener_tipo_produccion_actual(molino)
    necesidades = calcular_necesidades(molino, tipo_produccion)
    if molino == "MC2" and material in ["Puzolana_Humeda", "Puzolana_Seca"]:
        return necesidades.get("Puzolana_Humeda", 0) + necesidades.get("Puzolana_Seca", 0)
    return necesidades.get(material, 0)

def obtener_tipo_produccion_actual(molino):
    # Esta función debería obtener el tipo de producción actual del molino
    # Por ahora, usaremos un valor predeterminado
    if molino == "MC1":
        return "P30"
    elif molino == "MC2":
        return "P16"
    else:  # MC3
        return "P30"

def seleccionar_ruta_alimentacion(molino, material):
    if molino == "MC1":
        if material == "Clinker":
            return "Hacia MC1 desde Pretrit"
        elif material == "Puzolana":
            tiempo_vaciado_MC1 = calcular_tiempo_vaciado("MC1", "Puzolana")
            tiempo_vaciado_MC2 = calcular_tiempo_vaciado("MC2", "Puzolana_Humeda")
            return "Húmeda a MC1 por MC1" if tiempo_vaciado_MC1 < tiempo_vaciado_MC2 else "Húmeda a MC1 por MC2"
        elif material == "Yeso":
            tiempo_vaciado_MC1 = calcular_tiempo_vaciado("MC1", "Yeso")
            tiempo_vaciado_MC2 = calcular_tiempo_vaciado("MC2", "Yeso")
            return "Hacia MC1 por MC1" if tiempo_vaciado_MC1 < tiempo_vaciado_MC2 else "Hacia MC1 por MC2"
    elif molino == "MC2":
        if material == "Clinker":
            return "Hacia MC2 desde Pretrit"
        elif material == "Puzolana_Humeda":
            return "P.H. a 426HO04 por MC2"
        elif material == "Puzolana_Seca":
            return "P.S a 426HO02 por 426HO04"
        elif material == "Yeso":
            return "Hacia MC2 por MC2"
    elif molino == "MC3":
        if material == "Clinker":
            if niveles_MC3["Clinker_Silo_Blanco"] / tolvas_MC3["Clinker_Silo_Blanco"]["capacidad"] > 0.3:
                return "Hacia MC3 desde Silo3"
            else:
                return "Hacia Silo 3 desde Pretrit"
        elif material == "Puzolana":
            return "P. seca a MC3 por MC2"
        elif material == "Yeso":
            tiempo_vaciado_MC1 = calcular_tiempo_vaciado("MC1", "Yeso")
            tiempo_vaciado_MC2 = calcular_tiempo_vaciado("MC2", "Yeso")
            tiempo_vaciado_MC3 = calcular_tiempo_vaciado("MC3", "Yeso")
            if tiempo_vaciado_MC1 <= tiempo_vaciado_MC2 and tiempo_vaciado_MC1 <= tiempo_vaciado_MC3:
                return "Hacia MC3 por MC1"
            elif tiempo_vaciado_MC2 <= tiempo_vaciado_MC1 and tiempo_vaciado_MC2 <= tiempo_vaciado_MC3:
                return "Hacia MC3 por MC2"
            else:
                return "Hacia MC3 por MC3"
    return "Ruta no especificada"

def imprimir_tiempos(molino, tipo_produccion):
    necesidades = calcular_necesidades(molino, tipo_produccion)
    tiempos_llenado = calcular_tiempos_llenado(molino, necesidades)

    print(f"\nTiempos de llenado y vaciado para {molino} - {tipo_produccion}:")
    print("-" * 60)
    print(f"{'Material':<15}{'Tiempo Llenado (h)':<20}{'Tiempo Vaciado (h)':<20}")
    print("-" * 60)

    for material, tiempo_llenado in tiempos_llenado.items():
        tiempo_vaciado = calcular_tiempo_vaciado(molino, material)
        tiempo_llenado_str = f"{tiempo_llenado:.2f}" if tiempo_llenado != float('inf') else "∞"
        tiempo_vaciado_str = f"{tiempo_vaciado:.2f}" if tiempo_vaciado != float('inf') else "∞"
        print(f"{material:<15}{tiempo_llenado_str:<20}{tiempo_vaciado_str:<20}")

def generar_recomendaciones(molino_actual):
    recomendaciones = []
    restricciones = []

    def espacio_disponible(nivel_actual, capacidad, rendimiento):
        return capacidad * rendimiento - nivel_actual

    if molino_actual == "MC1":
        rendimiento = 0.8
        esp_clinker = espacio_disponible(niveles_MC1["Clinker"], tolvas_MC1["Clinker"]["capacidad"], rendimiento)
        esp_puzolana = espacio_disponible(niveles_MC1["Puzolana"], tolvas_MC1["Puzolana"]["capacidad"], rendimiento)
        esp_yeso = min(espacio_disponible(niveles_MC1["Yeso"], tolvas_MC1["Yeso"]["capacidad"], rendimiento), 60 - niveles_MC1["Yeso"])

        if esp_clinker > 0:
            recomendaciones.append(f"Recomiendo alimentar Clinker a MC1 desde Pretrit (Espacio disponible: {esp_clinker:.2f}t)")
        if esp_puzolana > 0:
            recomendaciones.append(f"Recomiendo alimentar Puzolana a MC1 (Espacio disponible: {esp_puzolana:.2f}t)")
        if esp_yeso > 0:
            recomendaciones.append(f"Recomiendo alimentar Yeso a MC1 (Espacio disponible: {esp_yeso:.2f}t)")

    elif molino_actual == "MC2":
        rendimiento = 0.8
        esp_clinker = espacio_disponible(niveles_MC2["Clinker"], tolvas_MC2["Clinker"]["capacidad"], rendimiento)
        esp_puzolana_h = espacio_disponible(niveles_MC2["Puzolana_Humeda"], tolvas_MC2["Puzolana_Humeda"]["capacidad"], rendimiento)
        esp_puzolana_s = espacio_disponible(niveles_MC2["Puzolana_Seca"], tolvas_MC2["Puzolana_Seca"]["capacidad"], rendimiento)
        esp_yeso = espacio_disponible(niveles_MC2["Yeso"], tolvas_MC2["Yeso"]["capacidad"], rendimiento)

        if esp_clinker > 0:
            recomendaciones.append(f"Recomiendo alimentar Clinker a MC2 desde Pretrit (Espacio disponible: {esp_clinker:.2f}t)")
        if esp_puzolana_h > 0:
            recomendaciones.append(f"Recomiendo alimentar Puzolana Húmeda a 426HO04 por MC2 (Espacio disponible: {esp_puzolana_h:.2f}t)")
        if esp_puzolana_s > 0:
            recomendaciones.append(f"Recomiendo alimentar Puzolana Seca a 426HO02 por 426HO04 (Espacio disponible: {esp_puzolana_s:.2f}t)")
        if esp_yeso > 0:
            recomendaciones.append(f"Recomiendo alimentar Yeso a MC2 por MC2 (Espacio disponible: {esp_yeso:.2f}t)")

    elif molino_actual == "MC3":
        rendimiento = 0.5
        esp_clinker = espacio_disponible(niveles_MC3["Clinker"], tolvas_MC3["Clinker"]["capacidad"], rendimiento)
        esp_clinker_sb = espacio_disponible(niveles_MC3["Clinker_Silo_Blanco"], tolvas_MC3["Clinker_Silo_Blanco"]["capacidad"], rendimiento)
        esp_puzolana = espacio_disponible(niveles_MC3["Puzolana"], tolvas_MC3["Puzolana"]["capacidad"], rendimiento)
        esp_yeso = espacio_disponible(niveles_MC3["Yeso"], tolvas_MC3["Yeso"]["capacidad"], rendimiento)

        if esp_clinker > 0:
            if niveles_MC3["Clinker_Silo_Blanco"] / tolvas_MC3["Clinker_Silo_Blanco"]["capacidad"] > 0.3:
                recomendaciones.append(f"Recomiendo alimentar Clinker a MC3 desde Silo Blanco (Espacio disponible: {esp_clinker:.2f}t)")
            else:
                recomendaciones.append(f"Recomiendo alimentar Clinker a Silo Blanco desde Pretrit (Espacio disponible: {esp_clinker_sb:.2f}t)")
        if esp_puzolana > 0:
            recomendaciones.append(f"Recomiendo alimentar Puzolana Seca a MC3 por MC2 (Espacio disponible: {esp_puzolana:.2f}t)")
        if esp_yeso > 0:
            recomendaciones.append(f"Recomiendo alimentar Yeso a MC3 (Espacio disponible: {esp_yeso:.2f}t)")

    if not recomendaciones:
        recomendaciones.append(f"Todas las tolvas de {molino_actual} están llenas o cerca de su capacidad máxima.")

    restricciones.append("Es necesario vaciar el sistema antes de una nueva alimentación para evitar contaminación.")
    restricciones.append("No se puede alimentar simultáneamente Clinker, Puzolana y Yeso a MC3.")
    restricciones.append("No se puede alimentar Clinker a múltiples molinos simultáneamente.")

    return recomendaciones, restricciones

# def optimizar_alimentacion(molino, tipo_produccion):
#     necesidades = calcular_necesidades(molino, tipo_produccion)
#     print(f"\nNecesidades para {molino} - {tipo_produccion}:")
#     for material, cantidad in necesidades.items():
#         print(f"{material}: {cantidad:.2f} t/h")

#     imprimir_tiempos(molino, tipo_produccion)

#     recomendaciones, restricciones = generar_recomendaciones(molino)
#     print("\nRecomendaciones:")
#     for recomendacion in recomendaciones:
#         print(recomendacion)

#     materiales_alimentados = []
#     for material, cantidad_necesaria in necesidades.items():
#         if cantidad_necesaria > 0:
#             ruta = seleccionar_ruta_alimentacion(molino, material)
#             print(f"\nIntentando alimentar {material} a {molino} por la ruta: {ruta}")
#             cantidad_alimentada = intentar_alimentar(molino, material, cantidad_necesaria)
#             if cantidad_alimentada > 0:
#                 materiales_alimentados.append(material)

#     print("\nRestricciones aplicables:")
#     restricciones_aplicables = [r for r in restricciones if any(mat in r for mat in materiales_alimentados)]
#     for restriccion in restricciones_aplicables:
#         print(restriccion)

#     print("\nTiempos actualizados después de la alimentación:")
#     imprimir_tiempos(molino, tipo_produccion)

# def main():
#     global niveles_MC1, niveles_MC2, niveles_MC3
#     ciclo = 1

#     while True:
#         print(f"\n--- Ciclo de optimización {ciclo} ---")

#         for molino in ["MC1", "MC2", "MC3"]:
#             if molino == "MC1":
#                 tipos_validos = ["P30", "P40"]
#             elif molino == "MC2":
#                 tipos_validos = ["P10", "P16", "P20", "P30"]
#             elif molino == "MC3":
#                 tipos_validos = ["P30"]

#             print(f"\nOptimizando {molino}")
#             print(f"Este molino puede producir: {', '.join(tipos_validos)}")
#             tipo_produccion = input(f"Ingrese el tipo de producción para {molino} (o presione Enter para omitir): ").upper()

#             if tipo_produccion:
#                 if tipo_produccion not in tipos_validos:
#                     print(f"Error, Ingrese un tipo de producción válido para {molino}")
#                     continue
#                 optimizar_alimentacion(molino, tipo_produccion)

#         print("\nEstado actual de las alimentaciones:")
#         for molino, materiales in alimentaciones_en_progreso.items():
#             if materiales:
#                 print(f"{molino}: {', '.join(materiales)}")
#             else:
#                 print(f"{molino}: Sin alimentaciones en progreso")

#         # Esperar a que todos los temporizadores terminen
#         while any(temporizadores_molinos.values()):
#             time.sleep(1)

#         print("\nTodas las alimentaciones han sido completadas.")
#         print("\nEstado actual después de la alimentación:")
#         print(f"Niveles MC1: {redondear_diccionario(niveles_MC1)}")
#         print(f"Niveles MC2: {redondear_diccionario(niveles_MC2)}")
#         print(f"Niveles MC3: {redondear_diccionario(niveles_MC3)}")

#         if todas_tolvas_llenas():
#             print("Todas las tolvas están llenas. Finalizando la simulación.")
#             break

#         ciclo += 1

#         continuar = input("¿Desea continuar con otro ciclo? (s/n): ").lower()
#         if continuar != 's':
#             break

#     print("Simulación completada.")

# if __name__ == "__main__":
#     main()

def verificar_alimentacion_completa(molino, tipo_produccion, niveles):
    necesidades = calcular_necesidades(molino, tipo_produccion)
    materias_faltantes = []

    for material, cantidad_necesaria in necesidades.items():
        if cantidad_necesaria > 0 and niveles[material] == 0:
            materias_faltantes.append(material)

    return materias_faltantes

def optimizar_alimentacion(molino, tipo_produccion):
    global niveles_MC1, niveles_MC2, niveles_MC3
    necesidades = calcular_necesidades(molino, tipo_produccion)
    print(f"\nNecesidades para {molino} - {tipo_produccion}:")
    for material, cantidad in necesidades.items():
        print(f"{material}: {cantidad:.2f} t/h")

    imprimir_tiempos(molino, tipo_produccion)

    recomendaciones, restricciones = generar_recomendaciones(molino)
    print("\nRecomendaciones:")
    for recomendacion in recomendaciones:
        print(recomendacion)

    materiales_alimentados = []
    for material, cantidad_necesaria in necesidades.items():
        if cantidad_necesaria > 0:
            ruta = seleccionar_ruta_alimentacion(molino, material)
            print(f"\nIntentando alimentar {material} a {molino} por la ruta: {ruta}")
            cantidad_alimentada = intentar_alimentar(molino, material, cantidad_necesaria)
            if cantidad_alimentada > 0:
                materiales_alimentados.append(material)

    print("\nRestricciones aplicables:")
    restricciones_aplicables = [r for r in restricciones if any(mat in r for mat in materiales_alimentados)]
    for restriccion in restricciones_aplicables:
        print(restriccion)

    print("\nTiempos actualizados después de la alimentación:")
    imprimir_tiempos(molino, tipo_produccion)

    # Verificar si todas las materias primas necesarias fueron alimentadas
    if molino == "MC1":
        niveles = niveles_MC1
    elif molino == "MC2":
        niveles = niveles_MC2
    else:  # MC3
        niveles = niveles_MC3

    materias_faltantes = verificar_alimentacion_completa(molino, tipo_produccion, niveles)
    if materias_faltantes:
        print(f"\nAdvertencia: No se pudo alimentar todas las materias primas necesarias para {molino}.")
        print(f"Materias primas faltantes: {', '.join(materias_faltantes)}")
        print("No se puede elaborar el producto con la configuración actual.")
    else:
        print(f"\nTodas las materias primas necesarias para {molino} han sido alimentadas correctamente.")
        print("El producto puede ser elaborado con la configuración actual.")

def main():
    global niveles_MC1, niveles_MC2, niveles_MC3
    ciclo = 1

    while True:
        print(f"\n--- Ciclo de optimización {ciclo} ---")

        for molino in ["MC1", "MC2", "MC3"]:
            if molino == "MC1":
                tipos_validos = ["P30", "P40"]
            elif molino == "MC2":
                tipos_validos = ["P10", "P16", "P20", "P30"]
            elif molino == "MC3":
                tipos_validos = ["P30"]

            print(f"\nOptimizando {molino}")
            print(f"Este molino puede producir: {', '.join(tipos_validos)}")
            tipo_produccion = input(f"Ingrese el tipo de producción para {molino} (o presione Enter para omitir): ").upper()

            if tipo_produccion:
                if tipo_produccion not in tipos_validos:
                    print(f"Error, Ingrese un tipo de producción válido para {molino}")
                    continue
                optimizar_alimentacion(molino, tipo_produccion)

        print("\nEstado actual de las alimentaciones:")
        for molino, materiales in alimentaciones_en_progreso.items():
            if materiales:
                print(f"{molino}: {', '.join(materiales)}")
            else:
                print(f"{molino}: Sin alimentaciones en progreso")

        # Esperar a que todos los temporizadores terminen
        while any(temporizadores_molinos.values()):
            time.sleep(1)

        print("\nTodas las alimentaciones en progreso han sido completadas.")
        print("\nEstado actual después de la alimentación:")
        print(f"Niveles MC1: {redondear_diccionario(niveles_MC1)}")
        print(f"Niveles MC2: {redondear_diccionario(niveles_MC2)}")
        print(f"Niveles MC3: {redondear_diccionario(niveles_MC3)}")

        if todas_tolvas_llenas():
            print("Todas las tolvas están llenas. Finalizando la simulación.")
            break

        ciclo += 1

        continuar = input("¿Desea continuar con otro ciclo? (s/n): ").lower()
        if continuar != 's':
            break

    print("Simulación completada.")

if __name__ == "__main__":
    main()


--- Ciclo de optimización 1 ---

Optimizando MC1
Este molino puede producir: P30, P40

Necesidades para MC1 - P30:
Clinker: 49.32 t/h
Puzolana: 21.60 t/h
Yeso: 1.08 t/h

Tiempos de llenado y vaciado para MC1 - P30:
------------------------------------------------------------
Material       Tiempo Llenado (h)  Tiempo Vaciado (h)  
------------------------------------------------------------
Clinker        8.11                -2.03               
Puzolana       11.11               -2.78               
Yeso           55.56               -55.56              

Recomendaciones:
Recomiendo alimentar Clinker a MC1 desde Pretrit (Espacio disponible: 400.00t)
Recomiendo alimentar Puzolana a MC1 (Espacio disponible: 240.00t)
Recomiendo alimentar Yeso a MC1 (Espacio disponible: 60.00t)

Intentando alimentar Clinker a MC1 por la ruta: Hacia MC1 desde Pretrit
Iniciando alimentación de 49.32t de Clinker a MC1
MC1 - Alimentando Clinker: [                                                  ] 0.0%
Inicia